<a href="https://colab.research.google.com/github/ArtikAzh/GameDataset/blob/main/GameDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, MaxAbsScaler, PowerTransformer, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


class GameIdeaAnalyzer:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.data = None
        self.model = None

    def load_data(self):
        try:
            self.data = pd.read_csv(self.dataset_path, delimiter=";", encoding='ISO-8859-1')
            #self.data = self.data.iloc[1:]
            #self.data.fillna(self.data.mean(), inplace=True)
            self.data.iloc[:, 4] = self.data.iloc[:, 4].str.replace("+", "").astype(int)
            print("\nPreprocessed Data:")
            print(self.data.head())
        except Exception as e:
            print(f"Error loading data: {e}")

    def preprocess_data(self):
        try:
            laben = LabelEncoder()
            self.data.iloc[:, 1] = laben.fit_transform(self.data.iloc[:, 1])
            #scaler = StandardScaler()
            scaler = MaxAbsScaler()
            #scaler = PowerTransformer()
            self.data.iloc[:, [1, 4]] = scaler.fit_transform(self.data.iloc[:, [1, 4]])
        except Exception as e:
            print(f"Error preprocessing data: {e}")


    def split_data(self):
        try:
            X = self.data.iloc[:, [1, 4]]
            y = self.data.iloc[:, 5]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.56)
            return X_train, X_test, y_train, y_test
        except Exception as e:
            print(f"Error splitting data: {e}")
            return None, None, None, None

    """
    def train_model(self, X_train, y_train):
        try:
            self.model = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=640, random_state=64)
            self.model.fit(X_train, y_train)
        except Exception as e:
            print(f"Error training model: {e}")

    def train_model(self, X_train, y_train):
        try:
            self.model = MLPClassifier(max_iter=100, random_state=64)
            param_grid = {
                'hidden_layer_sizes': [(100,100,100), (64,64,64), (100,)],
                'activation': ['tanh', 'relu'],
                'solver': ['sgd', 'adam'],
                'alpha': [0.0001, 0.05],
                'learning_rate': ['constant','adaptive'],
            }
            grid_search = GridSearchCV(self.model, param_grid, cv=3)
            grid_search.fit(X_train, y_train)
            self.model = grid_search.best_estimator_
            print(f"Best parameters found: {grid_search.best_params_}")
        except Exception as e:
            print(f"Error training model: {e}")
    """

    def train_model(self, X_train, y_train):
        try:
            self.model = MLPClassifier(max_iter=1000, random_state=64)
            param_distributions = {
                'hidden_layer_sizes': [(64,64,64), (64,100,64), (100,100,100)],
                'activation': ['tanh', 'relu'],
                'solver': ['sgd', 'adam'],
                'alpha': [0.025, 0.05],
                'learning_rate': ['constant','adaptive'],
            }
            CV = StratifiedKFold(n_splits=3)
            randomized_search = RandomizedSearchCV(self.model, param_distributions, n_iter=10, cv=CV, random_state=64)
            randomized_search.fit(X_train, y_train)
            self.model = randomized_search.best_estimator_
            print(f"\nBest parameters found: {randomized_search.best_params_}")
        except Exception as e:
            print(f"Error training model: {e}")

    def evaluate_model(self, X_test, y_test):
        try:
            y_pred = self.model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            f1 = f1_score(y_test, y_pred, average='weighted')
            print("\nPredicted Data:")
            print(y_pred)
            print("\nActual Data:")
            print(y_test.values)
            print(f"\nModel accuracy: {accuracy:.2f}")
            print(f"Precision: {precision:.2f}")
            print(f"Recall: {recall:.2f}")
            print(f"F1 Score: {f1:.2f}")
        except Exception as e:
            print(f"Error evaluating model: {e}")

    def save_model(self, model_path):
        try:
            joblib.dump(self.model, model_path)
            print(f"\nModel saved to {model_path}")
        except Exception as e:
            print(f"Error saving model: {e}")

    def load_model(self, model_path):
        try:
            self.model = joblib.load(model_path)
            print(f"\nModel loaded from {model_path}")
        except Exception as e:
            print(f"Error loading model: {e}")

    def display_processed_data(self):
        try:
            print("\nProcessed Data:")
            print(self.data.head())
        except Exception as e:
            print(f"Error displaying processed data: {e}")


if __name__ == "__main__":
    dataset_path = "/content/GameDataset.csv"
    model_path = "GameIdeaModel.pkl"
    analyzer = GameIdeaAnalyzer(dataset_path)
    analyzer.load_data()
    analyzer.preprocess_data()
    X_train, X_test, y_train, y_test = analyzer.split_data()
    if X_train is not None:
        analyzer.train_model(X_train, y_train)
        analyzer.evaluate_model(X_test, y_test)
        analyzer.save_model(model_path)
        analyzer.display_processed_data()